# Sistemas Inteligentes 2021/2022

## Mini-projeto 1: Pacman comilão

<img src="pacman.png" alt="Drawing" style="width: 100px;"/>

## Grupo: 42

### Elementos do Grupo

Número:  53687       Nome:   Ariana Dias   
Número:  53746       Nome:   Andrei Tataru  
Número:  51127       Nome:   Luís Ferreirinha  

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação dos estados

(descreva aqui, textualmente, como decidiu representar os estados em Python; ilustre nas células de código abaixo a representação em Python de um estado à sua escolha)

-  Estados  
    - Posição Pacman: $\left(x,y\right)$ onde $x,y=$ inteiros
    - Pontos: inteiro
    - Tempo: inteiro
    - Casas Visitadas: {(x,y) : n}, x,y,n = inteiros, n representa o número de visitas a uma casa
    - Conjunto das Pastilhas: {(x,y) : tipo}, x,y = inteiros, tipo é um char que representa o tipo da pastilha   
   
- Estado Inicial: [Posicao Pacmam, Pontos = 0, tempo = 0, Casas Vistadas = {}, Pastilhas]

- Goal Test: Pontos = M  

- Operadores:  
    - Mover-se nas quatro direcções ortogonais para as quatro células vizinhas:  
        - $(x,y) \to (x+i,y+j)$,  onde  $i,j = 0 \, \, ou \, \, 1$  
        - Se não for impedido pelos limites do mundo nem por obstáculos.
    - Comer a pastilha na casa, removendo esta e adicionando os respetivos pontos ao seu total de pontos:
        - Pontos Totais = Pontos Totais + Pontos da Pastilha
        - Se a casa onde o pacman se encontra contem uma pastilha


In [13]:
# se definiu uma classe para representar os estados, inclua aqui o código Python correspondente
class EspacoPacman():

    def __init__(self, Pos_Pacman : tuple[int, int], Pastilhas : dict, Visitas : dict):
        self.Pos = Pos_Pacman
        self.Pontos = 0
        self.Tempo = 0
        self.Pastilhas = Pastilhas

    def getEstado(self):
        return [self.Pos, self.Tempo, self.Pontos, self.Pastilhas]

    def setEstado(self, Pos, Tempo, Pontos, Pastilhas):
        self.Pos = Pos
        self.Pontos = Pontos
        self.Tempo = Tempo
        self.Pastilhas = Pastilhas


In [15]:
# representação de um estado à sua escolha

# Teste
pastilhas = {(1,1) : "N", (2,3) : "D", (0,2) : "C"}
Espaço = EspacoPacman((0,0), pastilhas)
print(Espaço.getEstado())
estado1 = Espaço.getEstado()

Espaço.setEstado((2,3),1,pastilhas)
estado2 = Espaço.getEstado()
print(estado2)

[(0, 0), 0, {(1, 1): 'N', (2, 3): 'D', (0, 2): 'C'}]
[(2, 3), 1, {(1, 1): 'N', (2, 3): 'D', (0, 2): 'C'}]


## Formulação do problema

(explique textualmente como decidiu formular em Python o seu problema)

In [1]:
from searchPlus import *

class PacmanPastilhas(Problem):
    
    mapa = []

    def __init__(self, pos, objetivo, pastilhas, obstaculos, dimensao):
        initial = EspaçoEstados(pos, pastilhas)
        super().__init__(initial, goal)

        # Iniciar Mapa
        mapa = [[0]*dimensao for x in range(dimensao)]
        for x, y in obstaculos:
            mapa[y][x] = 1

    def actions(self, state):
        pass

    def result(self, state, action):
        pass

    def path_cost(self, cost, state, action, next_state)
        pass

    def goal_test(self, estado):
        return estado[2] == self.goal

In [18]:
def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)

In [26]:
dimensao = 10
mapa = [[0]*dimensao for x in range(dimensao)]
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
fronteira = quadro(0,0,10)
obstaculos = fronteira | l | c
print(mapa)
for x,y in obstaculos:
    mapa[y][x] = 1
print()
print(mapa)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


## Criação de estados e do problema

(Mostrem que o código está a funcionar, construindo instâncias da classe **PacmanPastilhas**, fazendo display dos estados, verificando o teste do estado final, gerando as ações para alguns estados, executando ações a partir de alguns estados e gerando novos estados e mostrando a evolução dos custos; verificando que os estados não se modificam com as ações (são gerados novos estados) e que a igualdade e a comparação entre estados funciona. Mostrem que a execução de sequências de ações está a funcionar bem.)

In [ ]:
# código de teste do problema

def exec(p,estado,accoes):
    custo = 0
    for a in accoes:
        seg = p.result(estado,a)
        custo = p.path_cost(custo,estado,a,seg)
        estado = seg
    p.display(estado)
    print('Custo:',custo)
    print('Goal?',p.goal_test(estado))
    return (estado,custo)

## Teste de procura de solução

(utilização de algoritmos de procura aprendidos nas aulas e comparação dos resultados ao nível de tempo de execução e solução obtida; comente aqui os resultados obtidos e o que observa)

In [ ]:
# código de aplicação dos algoritmos